# transform Carrie's XML files to AMDocs template
First load powder(s) so we have a PID for them to be used in the build process docs.

Would be nice to be able to use saxonm as it support more of XSLT 2 for example.<br/>
Saxonica (https://www.saxonica.com/download/c.xml) would do so but is tricky to install on SciServer as it seems to require root privilege to put things in /usr/lib.<br/>
Currently unpacked in ~/workspace/nist/BIN/....
May be simpler to use Java libraries on beakerx container.

Will use lxml for now  which is ok for the XSLT 1 used here.

In [1]:
import lxml
import lxml.etree as ET
import os
import sys
import glob
import xml.dom.minidom
import xmlschema

In [2]:
from myconfig import CONFIG

In [16]:
AMBENCH=CONFIG.AMBENCH_URL.split("/")[2].split(".")[0]
EXCEL_FILE = CONFIG.SAMPLES_EXCEL_FILE
try : CONFIG.USER
except: CONFIG.USER = input('username: ')
try: CONFIG.PASS
except : CONFIG.PASS = getpass.getpass('enter password ')
AUTH=(CONFIG.USER, CONFIG.PASS)

In [17]:
sys.path.insert(0, CONFIG.pyUTILS_path)
from ambench.cdcs_utils import AMBench2022

In [18]:
try:
    ambench2022=AMBench2022(CONFIG.TEMPLATE,CONFIG.AMBENCH_URL,auth=AUTH)
except:
    ambench2022=None

xsd_filename=f'{CONFIG.XSD}AMDocs.xsd'
VALIDATOR=xmlschema.XMLSchema(xsd_filename,build=False)
VALIDATOR.build()
VALIDATOR.validity 

'valid'

## define functions

In [29]:
def xsd_validate(xmlfile):
    v=VALIDATOR.is_valid(xmlfile)
    if not(v):
        try:
            VALIDATOR.validate(xmlfile)
        except Exception as e:
            print(xmlfile," is not valid\n\n",e,"\n=====\n")
        return False
    else:
        print(xmlfile,' is valid')
        return True
    
# read XSLT files, create transformers for build process and powder
xsl_filename='AMBuildProcess.xsl'
xslt = ET.parse(xsl_filename)
transform = ET.XSLT(xslt)

xsl_filename='AMPowder.xsl'
xslt = ET.parse(xsl_filename)
powder_transform = ET.XSLT(xslt)

In [25]:
def map_powder(dom):
    # dom contains DOM (from lxml.etree.parse) of Carrie's XML document
    elems=dom.findall('./PowderInformation/SupplierInformation/PowderLot')
    if len(elems) == 1:
        lotnumber=elems[0].text
        PID=find_powder_for_lotnumber(lotnumber)
        if PID is None:
            PID=''
            is_new=True
        else:
            is_new=False
        powderXML = powder_transform(dom,PID=ET.XSLT.strparam(PID))
        powderfile=f'{POWDER_OUT}{lotnumber}.xml'
        try:
            powderXML.write(powderfile,xml_declaration=False, encoding='UTF-8',pretty_print=True)
            print("written powder file",powderfile)
            if VALIDATOR.is_valid(powderfile):
                return (lotnumber,(powderfile,is_new))
            else:
                print("XSD validation Error in",xml_filename,"\n")
                return None
        except Exception as e:
            print("Error in",xml_filename,"\n",e)
    else:
        print('cannot find lotnumber for powder in',xml_filename,'no file written')
    return None

In [19]:
def map_buildprocess(dom):
    elems=dom.findall('./PowderInformation/SupplierInformation/PowderLot')
    if len(elems) == 1:
        lotnumber=elems[0].text
        powderPID=find_powder_for_lotnumber(lotnumber)
    else:
        powderPID=None
    elems=dom.findall('./SampleID/name')
    if len(elems) == 1:
        buildplateId=elems[0].text
    buildplatePID='UNKNOWN'
    if ambench2022 is not None:
        buildplate=ambench2022.query_buildplate_amdoc(name=buildplateId)
        if buildplate is not None:
            buildplatePID=buildplate.pid
        else:
            print("No buildplate found for buildplateid=",buildplateId,"referenced in",xml_filename)
    # TODO find if buildprocess exists for buildPLate, then retrieve its PID
    PID=find_buildprocess_for_buildplatePID(buildplatePID)
    if PID is not None:
        is_new=False
    else:
        PID=''
        is_new = True

    tXML = transform(dom,PID=ET.XSLT.strparam(PID),BuildPlatePID=ET.XSLT.strparam(buildplatePID),PowderPID=ET.XSLT.strparam(powderPID))
    buildprocess_file=f'{BUILSPROCESS_OUT}AMBuildProcess-{os.path.basename(xml_filename)}'
    tXML.write(buildprocess_file, xml_declaration=False, encoding='UTF-8',pretty_print=True)
    if VALIDATOR.is_valid(buildprocess_file):
        return buildprocess_file,is_new
    else:
        print("XSD validation Error in",xml_filename,"\n",e)
        return None

In [20]:
def find_buildprocess_for_buildplatePID(buildplatePID):
    # could be made more efficient, finding all processes in one query, create dict etc etc
    MQ={"AMDoc.AMBuildProcess.buildPlateID":buildplatePID}
    df=ambench2022.mongo_query(MQ)
    if len(df) == 1:
        dom = ET.parse(df['xml_content'][0])
        elems=dom.findall('./pid')
        if len(elems) == 1:
            return elems[0].text
    elif len(df)>1:
        raise Exception("Multiple build processes defined for buildplatePID",buildplatePID)
    return None

def find_powder_for_lotnumber(lotnumber):
    MQ={"AMDoc.AMPowder.lotNumber":lotnumber}
    df=ambench2022.mongo_query(MQ)
    if len(df) == 1:
        _xml=df['xml_content'][0]
        dom = ET.fromstring(bytes(_xml, encoding='utf-8'))
#         dom = ET.fromstring(df['xml_content'][0])
        elems=dom.findall('./pid')
        if len(elems) == 1:
            return elems[0].text
    elif len(df)>1:
        raise Exception("Multiple powders defined for lotnumber",lotnumber)
    return None


In [21]:
def load_amdocs_cdcs(amdocs):
    uploaded={}
    errors={}
    for f,is_new in amdocs:
        fn=os.path.basename(f)
        if is_new:
            print('upload new:', f)
            response=ambench2022.upload_data(f)
        else:
            print('update existing:', f)
            response=ambench2022.update_data(f)
        if response.ok:
            uploaded[fn]=response.json()    
        else:
            errors[fn]=response.json()
    return uploaded,errors

In [22]:
CARRIE="/home/idies/workspace/AMBench/DATA/CDCS/AltXML/carrie/"
carries_files=glob.glob(f'{CARRIE}AMBench2022*.xml')

AMDOC_OUT="/home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/"
POWDER_OUT=f'{AMDOC_OUT}AMPowder/'
os.makedirs(POWDER_OUT,exist_ok=True)
BUILSPROCESS_OUT=f'{AMDOC_OUT}AMBuildProcess/'
os.makedirs(BUILSPROCESS_OUT,exist_ok=True)

## First AMPowder-s

In [30]:
t_files=glob.glob(f'{POWDER_OUT}*.xml')
for file in t_files:
    os.remove(file)

powder_amdocs={}
for xml_filename in carries_files:
    try:
        dom = ET.parse(xml_filename)
        powder=map_powder(dom)
        if powder is not None:
            powder_amdocs[powder[0]]=powder[1]
    except Exception as ex:
        print("XML Syntax error\n",ex)
        raise(ex)

written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml
written powder file /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml


In [33]:
load_result = load_amdocs_cdcs(powder_amdocs.values())

update existing: /home/idies/workspace/AMBench/DATA/CDCS/AltXML/amdoc/AMPowder/K201801.xml


## then AMBuildProcess

In [ ]:
t_files=glob.glob(f'{BUILSPROCESS_OUT}*.xml')
for file in t_files:
    os.remove(file)

amdocs=[]
for xml_filename in sorted(carries_files):
    try:
        dom = ET.parse(xml_filename)
        bp=map_buildprocess(dom)
        if bp is not None:
            amdocs.append(bp)
        else:
            print("no result for",xml_filename)
    except Exception as ex:
        print("XML Syntax error\n",ex)
        pass

In [ ]:
for xmlfile,is_new in amdocs:
    v=VALIDATOR.is_valid(xmlfile)
    if not(v):
        try:
            VALIDATOR.validate(xmlfile)
        except Exception as e:
            print(xmlfile,"\n",e,"\n=====\n")
            raise e
    else:
        print(xmlfile,"is valid")

In [ ]:
load_result = load_amdocs_cdcs(amdocs)

## merging Carries and Barndons file
then xslt on combined document

In [ ]:
f_brandon='/home/idies/workspace/AMBench/DATA/CDCS/AltXML/brandon/AMB2022-01-718-AMMT-B6_ExampleBuildMetadata_v0.xml'
f_carrie='/home/idies/workspace/AMBench/DATA/CDCS/AltXML/carrie/AMBench2022-IN718-AMMT-B6.xml'
carrie, brandon = et.parse(f_carrie), et.parse(f_brandon)

In [ ]:
c=carrie.getroot()
b=brandon.getroot()
c.append(b)
f=f'/home/idies/workspace/AMBench/DATA/CDCS/AltXML/cb/cb-B6.xml'
carrie.write(f)

In [ ]:

xsd_filename='/home/idies/workspace/AMBench/DATA/CDCS/AltXML/brandon/am_schema_R2019a (Version 1).xsd'
xsd_filename='/home/idies/workspace/AMBench/DATA/CDCS/AltXML/brandon/am_schema_R2021_test0 (Version 1).xsd'
# xsd_filename='/home/idies/workspace/AMBench/DATA/CDCS/AltXML/brandon/AMMD-AMB2022.xsd'
AMMT_VALIDATOR=xmlschema.XMLSchema(xsd_filename,build=False)
AMMT_VALIDATOR.build()
AMMT_VALIDATOR.validity 

In [ ]:
AMMT_VALIDATOR.validate(f_brandon)